In [2]:
import pandas as pd
import geopandas as gpd
import os
from osgeo import ogr
import numpy as np

#### Global treatment of Eurostat and REMIND 

##### Global steps :
- Use the saved files from Eurostat 
- Distribute shares for countries to the total value
- Save new final output file with only relevant data

In [3]:
os.chdir(r"C:\Users\mar.eco\OneDrive - CBS - Copenhagen Business School\Desktop\REMIND")
transport_data = pd.read_csv("REMIND_Transport_Output.txt", delimiter='\t')
transport_data = transport_data.drop(columns={"Model", "Scenario","2055", "2060", "2070", "2080", "2090", "2100", "2110", "2130", "2150"})
transport_data.Region.unique()
Variable = transport_data['Variable'].str.split('|', expand=True)

In [4]:
transport_EU = transport_data.query("Region == 'EU27'")

In [5]:
def split_variable(df):
    variables = df['Variable'].str.split('|', expand=True)
    num_cols =  variables.shape[1]
    new_column_names = [f'Variable_{i}' for i in range(num_cols)]
    df[new_column_names] = variables
    return(df)

In [6]:
transport_EU = split_variable(transport_EU)

C:\Users\mar.eco\AppData\Local\Temp\ipykernel_14988\3017919102.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_column_names] = variables
C:\Users\mar.eco\AppData\Local\Temp\ipykernel_14988\3017919102.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_column_names] = variables
C:\Users\mar.eco\AppData\Local\Temp\ipykernel_14988\3017919102.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [7]:
final_energy = transport_EU[(transport_EU["Variable_0"] == "FE") & (transport_EU["Variable_1"] == "Transport")]

In [8]:
final_energy

,Region,Variable,Unit,2005,2010,2015,2020,2025,2030,2035,...,Variable_3,Variable_4,Variable_5,Variable_6,Variable_7,Variable_8,Variable_9,Variable_10,Variable_11,Variable_12
82801,EU27,FE|Transport|Bunkers|Freight|International Shi...,EJ/yr,0.009081388136571,0.0165694857709562,0.0266747003322437,0.0862549179835251,0.175209537131761,0.175696922793722,0.182369826095918,...,Freight,International Shipping,Liquids,Biomass,None,None,None,None,None,None
82826,EU27,FE|Transport|Bunkers|Freight|International Shi...,EJ/yr,192.822.999.825.547,190.216.679.908.231,188.729.470.240.778,182.864.640.100.136,173.551.959.036.235,172.096.859.194.787,170.025.701.602.085,...,Freight,International Shipping,Liquids,Fossil,None,None,None,None,None,None
82851,EU27,FE|Transport|Bunkers|Freight|International Shi...,EJ/yr,0,0,0,0,0,0,0,...,Freight,International Shipping,Liquids,Hydrogen,None,None,None,None,None,None
82876,EU27,FE|Transport|Bunkers|Pass|International Aviati...,EJ/yr,0.0130635639285938,0.0211886225501252,0.0282663003465992,0.0667792098012143,0.11347173787013,0.112373860519604,0.114466759616053,...,Pass,International Aviation,Liquids,Biomass,None,None,None,None,None,None
82901,EU27,FE|Transport|Bunkers|Pass|International Aviati...,EJ/yr,124.886.037.637.166,128.644.349.186.318,121.614.445.756.805,114.099.687.205.296,107.328.472.491.565,104.698.203.561.295,10.111.123.061.252,...,Pass,International Aviation,Liquids,Fossil,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135672,EU27,FE|Transport|Pass|Aviation|Hydrogen,EJ/yr,0,0,0,0,"1,97E+06","4,83E+06","1,14E+07",...,Aviation,Hydrogen,None,None,None,None,None,None,None,None
135675,EU27,FE|Transport|Pass|Aviation|Liquids,EJ/yr,15.358.660.568.302,157.681.482.839.692,137.019.891.867.888,134.341.835.065.713,132.800.468.389.527,133.867.745.431.757,132.624.061.906.371,...,Aviation,Liquids,None,None,None,None,None,None,None,None
135799,EU27,FE|Transport|Pass|Aviation|Liquids||Fossil,EJ/yr,152.013.601.358.476,155.112.711.268.789,133.898.158.035.284,126.738.389.444.875,120.111.216.225.314,120.665.096.100.689,118.356.524.887.069,...,Aviation,Liquids,,Fossil,None,None,None,None,None,None
135802,EU27,FE|Transport|Pass|Aviation|Liquids||Hydrogen,EJ/yr,0,0,0,0,0,0,0,...,Aviation,Liquids,,Hydrogen,None,None,None,None,None,None


In [9]:
def filter_exact_path(df, path):
    variables_split = df['Variable'].str.split('|', expand=True)
    expected_depth = len(path.split('|'))
    
    # Replace None/NaN with empty strings to avoid errors
    cleaned = variables_split.iloc[:, :expected_depth].fillna('')
    
    joined_path = cleaned.agg('|'.join, axis=1)
    actual_depth = variables_split.notna().sum(axis=1)

    mask = (joined_path == path) & (actual_depth == expected_depth)
    return df[mask]

In [10]:
categories = [
    "FE|Transport|Freight|Road|Heavy",
    "FE|Transport|Freight|Road|Light",
    "FE|Transport|Pass|Road|Bus",
    "FE|Transport|Pass|Road|LDV|Four Wheelers",
    "FE|Transport|Pass|Road|LDV|Two Wheelers",
    "FE|Transport|Pass|Domestic Aviation",
    "FE|Transport|Pass|Aviation",
    "FE|Transport|Pass|Rail",
    "FE|Transport|Freight|Rail"
]

In [11]:
categories_fuel_4w = [
    "FE|Transport|Pass|Road|LDV|Four Wheelers|Electricity",
    "FE|Transport|Pass|Road|LDV|Four Wheelers|Hydrogen",
    "FE|Transport|Pass|Road|LDV|Four Wheelers|Gases",
    "FE|Transport|Pass|Road|LDV|Four Wheelers|Liquids",
]

In [12]:
filtered_data = {}
for category in categories:
    filtered_data[category] = filter_exact_path(final_energy, category)

In [13]:
filtered_data_4w = {}
for category in categories_fuel_4w:
    filtered_data_4w[category] = filter_exact_path(final_energy, category)

In [14]:
filtered_data['FE|Transport|Pass|Road|LDV|Four Wheelers']

,Region,Variable,Unit,2005,2010,2015,2020,2025,2030,2035,...,Variable_3,Variable_4,Variable_5,Variable_6,Variable_7,Variable_8,Variable_9,Variable_10,Variable_11,Variable_12
106996,EU27,FE|Transport|Pass|Road|LDV|Four Wheelers,EJ/yr,640.226.534.352.189,678.462.004.864.356,675.586.664.220.426,675.642.308.227.177,61.789.786.563.611,52.929.170.576.245,397.189.409.064.838,...,Road,LDV,Four Wheelers,None,None,None,None,None,None,None


In [15]:
filtered_data_4w['FE|Transport|Pass|Road|LDV|Four Wheelers|Hydrogen']

,Region,Variable,Unit,2005,2010,2015,2020,2025,2030,2035,...,Variable_3,Variable_4,Variable_5,Variable_6,Variable_7,Variable_8,Variable_9,Variable_10,Variable_11,Variable_12
119211,EU27,FE|Transport|Pass|Road|LDV|Four Wheelers|Hydrogen,EJ/yr,0.0180263622107814,0.0186624675891378,0.016835180643571,0.0148032167074042,0.0180826180751206,0.0241812337937173,0.0614839109474022,...,Road,LDV,Four Wheelers,Hydrogen,None,None,None,None,None,None


##### Import Eurostat files to use as proxy  

The Eurostat files have share for every EU27 countries as well as the total value of EU27.
The objective is to create a new final dataframe that has :
- Transport category (12 total categories)
- Given EU27 value of REMIND for each year
- Share of the choesen Eurostat year for each country
- Values for each country of EU27 and each year 

In [16]:
file_names = ["estat_road_go_ta_mplw_heavy.tsv.gz", "estat_road_go_ta_mplw_light.tsv.gz", 
             "estat_tran_r_vehst.tsv.gz", "estat_road_eqs_carmot.tsv.gz", "estat_road_eqs_two_wheelers.tsv.gz",
              "estat_avia_panc.tsv.gz", "estat_avia_international.tsv.gz", 
              "estat_ttr00015.tsv.gz", "estat_ttr00006.tsv.gz"]

In [17]:
file_names_4w = ["estat_road_eqs_carmot.tsv.gz", "estat_road_eqs_carmot.tsv.gz", 
             "estat_road_eqs_carmot.tsv.gz", "estat_road_eqs_carmot.tsv.gz"]

In [18]:
base_path = r"C:\Users\mar.eco\OneDrive - CBS - Copenhagen Business School\Desktop\REMIND\Data\Final"

In [19]:
category_file_map = dict(zip(categories, file_names))
category_file_map

{'FE|Transport|Freight|Road|Heavy': 'estat_road_go_ta_mplw_heavy.tsv.gz',
 'FE|Transport|Freight|Road|Light': 'estat_road_go_ta_mplw_light.tsv.gz',
 'FE|Transport|Pass|Road|Bus': 'estat_tran_r_vehst.tsv.gz',
 'FE|Transport|Pass|Road|LDV|Four Wheelers': 'estat_road_eqs_carmot.tsv.gz',
 'FE|Transport|Pass|Road|LDV|Two Wheelers': 'estat_road_eqs_two_wheelers.tsv.gz',
 'FE|Transport|Pass|Domestic Aviation': 'estat_avia_panc.tsv.gz',
 'FE|Transport|Pass|Aviation': 'estat_avia_international.tsv.gz',
 'FE|Transport|Pass|Rail': 'estat_ttr00015.tsv.gz',
 'FE|Transport|Freight|Rail': 'estat_ttr00006.tsv.gz'}

In [20]:
category_file_map_4w = dict(zip(categories_fuel_4w, file_names_4w))
category_file_map_4w

{'FE|Transport|Pass|Road|LDV|Four Wheelers|Electricity': 'estat_road_eqs_carmot.tsv.gz',
 'FE|Transport|Pass|Road|LDV|Four Wheelers|Hydrogen': 'estat_road_eqs_carmot.tsv.gz',
 'FE|Transport|Pass|Road|LDV|Four Wheelers|Gases': 'estat_road_eqs_carmot.tsv.gz',
 'FE|Transport|Pass|Road|LDV|Four Wheelers|Liquids': 'estat_road_eqs_carmot.tsv.gz'}

In [21]:
def process_value(value):
    # Check if there's exactly one dot (then it is a decimal and not thousand)
    if value.count('.') == 1:
        if len(value.split('.')[0]) > 3:
            return value.replace(".", "")  # Remove dot as thousands separator
        else:
            return value
    else:
        # If there's no dot or multiple dots, remove all dots (treat as thousands)
        return value.replace(".", "")

In [22]:
all_distributed = []
for category, eurostat_file in category_file_map.items():
    remind_df = filtered_data[category]
    eurostat_path = os.path.join(base_path, eurostat_file)

    remind_row = remind_df[remind_df["Region"] == "EU27"].iloc[0]
    years = [col for col in remind_row.index if col.isdigit()]
    
    remind_values = remind_row[years].astype(str).apply(process_value).astype(float)

    euro_df = pd.read_csv(eurostat_path, compression="gzip", delimiter="\t")
    euro_df.columns = euro_df.columns.str.strip()

    # Find share column = last one
    share_col = euro_df.columns[-1]
    shares = euro_df[["country_code", share_col]].copy()
    shares.columns = ["Country", "Share"]

    for year in years:
        eu_value = remind_values[year]
        shares["Value"] = shares["Share"] * eu_value
        shares["Year"] = year
        shares["Category"] = category

        all_distributed.append(shares[["Category", "Country", "Year", "Value"]])

In [23]:
all_distributed = []
for category, eurostat_file in category_file_map_4w.items():
    remind_df = filtered_data_4w[category]
    eurostat_path = os.path.join(base_path, eurostat_file)

    remind_row = remind_df[remind_df["Region"] == "EU27"].iloc[0]
    years = [col for col in remind_row.index if col.isdigit()]
    
    remind_values = remind_row[years].astype(str).apply(process_value).astype(float)

    euro_df = pd.read_csv(eurostat_path, compression="gzip", delimiter="\t")
    euro_df.columns = euro_df.columns.str.strip()

    # Find share column = last one
    share_col = euro_df.columns[-1]
    shares = euro_df[["country_code", share_col]].copy()
    shares.columns = ["Country", "Share"]

    for year in years:
        eu_value = remind_values[year]
        shares["Value"] = shares["Share"] * eu_value
        shares["Year"] = year
        shares["Category"] = category

        all_distributed.append(shares[["Category", "Country", "Year", "Value"]])

In [24]:
final_df = pd.concat(all_distributed, ignore_index=True)

In [25]:
final_df_4w = pd.concat(all_distributed, ignore_index=True)

In [26]:
pivoted_df = final_df.pivot_table(
    index=["Category", "Country"],  # rows: one per category & country
    columns="Year",                 # columns become the years
    values="Value"                  
).reset_index()

In [27]:
pivoted_df = final_df_4w.pivot_table(
    index=["Category", "Country"],  # rows: one per category & country
    columns="Year",                 # columns become the years
    values="Value"                  
).reset_index()

In [28]:
final_df[(final_df['Category'] == "FE|Transport|Freight|Rail") & (final_df['Year'] == "2020")]

,Category,Country,Year,Value


In [32]:
final_df_4w[(final_df_4w['Category'] == "FE|Transport|Pass|Road|LDV|Four Wheelers|Hydrogen") & (final_df['Year'] == "2020")]

,Category,Country,Year,Value
364,FE|Transport|Pass|Road|LDV|Four Wheelers|Hydrogen,AT,2020,0.000300
365,FE|Transport|Pass|Road|LDV|Four Wheelers|Hydrogen,BE,2020,0.000350
366,FE|Transport|Pass|Road|LDV|Four Wheelers|Hydrogen,BG,2020,0.000174
367,FE|Transport|Pass|Road|LDV|Four Wheelers|Hydrogen,CY,2020,0.000036
368,FE|Transport|Pass|Road|LDV|Four Wheelers|Hydrogen,CZ,2020,0.000376
369,FE|Transport|Pass|Road|LDV|Four Wheelers|Hydrogen,DE,2020,0.002838
370,FE|Transport|Pass|Road|LDV|Four Wheelers|Hydrogen,DK,2020,0.000163
371,FE|Transport|Pass|Road|LDV|Four Wheelers|Hydrogen,EE,2020,0.000050
372,FE|Transport|Pass|Road|LDV|Four Wheelers|Hydrogen,EL,2020,0.000340
373,FE|Transport|Pass|Road|LDV|Four Wheelers|Hydrogen,ES,2020,0.001548


In [23]:
check_sum = final_df[final_df['Category'] == "FE|Transport|Freight|Rail"].groupby("Year")["Value"].sum()
check_sum

Year
2005    0.243391
2010    0.236970
2015    0.198990
2020    0.243866
2025    0.250085
2030    0.250159
2035    0.241598
2040    0.240978
2045    0.239314
2050    0.237722
Name: Value, dtype: float64

In [24]:
output_path = r"C:\Users\mar.eco\OneDrive - CBS - Copenhagen Business School\Desktop\REMIND\Results_REMIND_yearly.csv"
final_df.to_csv(output_path, index=False)

In [39]:
output_path = r"C:\Users\mar.eco\OneDrive - CBS - Copenhagen Business School\Desktop\REMIND\Results_REMIND_4w_yearly.csv"
final_df_4w.to_csv(output_path, index=False)